In [36]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import re
import json
import unicodedata
import numpy as np
import ast
from sklearn.impute import KNNImputer
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../src'))

from data_preparation_utils import load_data, drop_and_save_columns, clean_text, filter_rows_by_column_value
pd.set_option('display.max_columns', None)

# load/save data

In [83]:
# Load processed data
df = load_data(r"G:\My Drive\french_resturants\data\processed\french_resturants_only.csv")

In [171]:
#save data frame
df.to_csv(r"G:\My Drive\french_resturants\data\processed\french_resturants_only.csv", index=False)

# finding columns with nulls

In [47]:
# Step 1: Find columns with at least one null
null_columns = df.columns[df.isnull().any()].tolist()

# Step 2: Create a subset DataFrame with only those columns
df_nulls = df[null_columns]

# Step 3: (Optional) Filter rows where at least one of those columns is actually null
df_nulls_only = df_nulls[df_nulls.isnull().any(axis=1)]

# Display summary
print(f"Found {len(null_columns)} columns with nulls.")
print(f"Subset has {df_nulls_only.shape[0]} rows with at least one null value.")

Found 47 columns with nulls.
Subset has 155223 rows with at least one null value.


In [50]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()
viz = AV.AutoViz(
    '',  # empty string means use a DataFrame directly
    dfte=df_nulls,
    depVar='',  # no target variable at this stage
)

    Since nrows is smaller than dataset, loading random sample of 150000 rows into pandas...
Shape of your Data Set loaded: (150000, 47)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  5
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  33
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  3
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  2
    Number of NLP String Columns =  4
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  0
    47 Predictors classified...
        No variables removed since no ID or low-information variables found in data s

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
region,object,0.001355,0,,,"2 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
province,object,11.351516,0,,,"16752 missing values. Impute them with mean, median, mode, or a constant value such as 123., 58 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: object, float,"
city,object,0.082670,9,,,"122 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
claimed,object,0.113163,0,,,"167 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
awards,object,71.663900,0,,,"105758 missing values. Impute them with mean, median, mode, or a constant value such as 123., 53 rare categories: Too many to list. Group them into a single category or drop the categories., Mixed dtypes: has 2 different data types: float, object,"
top_tags,object,7.556158,5,,,"11151 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
price_level,object,17.868880,0,,,"26370 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
price_range,object,65.751652,2,,,"97033 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"
meals,object,33.059122,0,,,"48787 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: float, object,, Possible high cardinality column with 399 unique values: Use hash encoding or text embedding to reduce dimension."
cuisines,object,13.267152,8,,,"19579 missing values. Impute them with mean, median, mode, or a constant value such as 123., Mixed dtypes: has 2 different data types: object, float,"


Number of All Scatter Plots = 15


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\idosc\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

Could not draw wordcloud plot for city. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

Could not draw wordcloud plot for top_tags. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.

Could not draw wordcloud plot for cuisines. 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://n

# nulls groups:

## ✅ Group A — Low Null % (under ~10%)

In [ ]:
# region

# city

# claimed

# moved_location

## 🟡 Group B — Moderate Null % (~10% to ~35%)

In [ ]:
# province

# top_tags

# cuisines

# meals

# default_language

## 🔴 Group C — High Null % (Over 50%)

In [ ]:
# awards

# special_diets

# features

# price_range

# mon_open_1st → sun_close_2nd (all time slot columns)

## 📈 Group D — Numerical Columns

In [ ]:
# open_days_per_week

# open_hours_per_week

# working_shifts_per_week

# rank

# total_restaurants_in_city

# null handling in group A

In [51]:
group_a_cols = ['region', 'city', 'claimed', 'moved_location']

for col in group_a_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# null handling in group B

In [54]:
# 📌 Group B: Moderate-null categorical columns
# Strategy:
# - Use "Unknown" for single-value categorical fields
# - Use "Missing" for multi-label/tag-like columns that will be encoded later

# Define how to fill each column
group_b_fill_map = {
    'province': 'Unknown',          # Discrete category
    'default_language': 'Unknown',  # Discrete category
    'top_tags': 'Missing',          # Multi-label (comma-separated tags)
    'cuisines': 'Missing',          # Multi-label (e.g., "French, Italian")
    'meals': 'Missing'              # Multi-label (e.g., "Lunch, Dinner")
}

# Apply the fill per column
for col, fill_value in group_b_fill_map.items():
    if col in df.columns:
        df[col] = df[col].fillna(fill_value)

# null handling in group C

## handling shifts columns

In [58]:
# Get all time slot columns
time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# Step 1: Identify rows where ALL time columns are null
all_time_slots_null = df[time_slot_cols].isnull().all(axis=1)

# Step 2: Fill partial nulls with "Closed" for others
for day in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
    open_1st = f'{day}_open_1st'
    close_1st = f'{day}_close_1st'
    open_2nd = f'{day}_open_2nd'
    close_2nd = f'{day}_close_2nd'
    
    # Only proceed if the columns exist
    if all(col in df.columns for col in [open_1st, close_1st, open_2nd, close_2nd]):

        # Condition: row is NOT fully empty in time columns
        not_all_null = ~all_time_slots_null

        # Handle 1st shift nulls
        df.loc[not_all_null & df[open_1st].isnull(), open_1st] = 'Closed'
        df.loc[not_all_null & df[close_1st].isnull(), close_1st] = 'Closed'

        # Handle 2nd shift nulls — only fill if it’s not "continuous"
        is_continuous = df[open_2nd] == 'continuous'
        df.loc[not_all_null & df[open_2nd].isnull() & ~is_continuous, open_2nd] = 'Closed'
        df.loc[not_all_null & df[close_2nd].isnull() & ~is_continuous, close_2nd] = 'Closed'

In [60]:
df[time_slot_cols].head(30)

,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,09:00,14:30,continuous,continuous,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,22:00,09:00,14:30,19:00,22:00,09:00,16:00,continuous,continuous
6,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00
7,Closed,Closed,Closed,Closed,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous
8,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:00,continuous,continuous,12:00,14:00,continuous,continuous,12:00,14:00,19:00,21:00,19:00,21:00,continuous,continuous,12:00,14:00,19:00,21:00
9,Closed,Closed,Closed,Closed,09:00,16:00,continuous,continuous,09:00,16:00,continuous,continuous,09:00,21:00,continuous,continuous,09:00,21:00,continuous,continuous,16:45,23:45,continuous,continuous,09:00,17:00,continuous,continuous


### KNN imputer - group by: region, province, city

In [85]:
# Count the rows with any null values in the time slot columns
null_rows_count = df[time_slot_cols].isnull().any(axis=1).sum()
print(f"Number of rows with null values in time slot columns in df: {null_rows_count}")

Number of rows with null values in time slot columns in df: 69849


In [87]:
# 1️⃣ Define the time slot columns (those starting with day abbreviations)
time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# 2️⃣ Convert each time slot column to categorical, store the mapping, and convert to numeric codes
category_maps = {}
for col in time_slot_cols:
    df[col] = df[col].astype('category')
    # Store the mapping: code -> original string
    category_maps[col] = dict(enumerate(df[col].cat.categories))
    # Convert to numeric codes; missing values become -1, so we replace those with NaN
    df[col] = df[col].cat.codes.replace(-1, np.nan)

# 3️⃣ Prepare a container to collect the imputed results per group
imputed_parts = []

# 4️⃣ Group by 'region', 'city', and 'province' and impute within each group
for group_key, group_df in df.groupby(['region', 'city', 'province']):
    # Extract the subset of time slot columns for the current group
    group_time_data = group_df[time_slot_cols]
    
    # Check if the entire group is missing (i.e. all values in group_time_data are NaN)
    if group_time_data.isnull().all().all():
        # Leave the group untouched by appending the original (all-NaN) data
        imputed_parts.append(group_time_data)
    else:
        # Determine the number of neighbors as the smaller of 3 or the group size
        n_neighbors = min(3, len(group_df))
        imputer = KNNImputer(n_neighbors=n_neighbors)
        # Apply KNN imputation on the group's time data
        imputed_array = imputer.fit_transform(group_time_data)
        # Create a DataFrame from the imputed result with the same columns and index as the group
        imputed_df = pd.DataFrame(imputed_array, columns=time_slot_cols, index=group_df.index)
        imputed_parts.append(imputed_df)

# 5️⃣ Recombine all imputed groups into one DataFrame, sorted by index
df_imputed_first_grouping = pd.concat(imputed_parts).sort_index()

# 6️⃣ Decode the numeric codes back to the original string values using the stored mappings
for col in time_slot_cols:
    mapping = category_maps[col]
    df_imputed_first_grouping[col] = df_imputed_first_grouping[col].round().astype('Int64').map(mapping)

print("✅ First grouping imputation (by region, city, province) completed.")

# 7️⃣ Update the original DataFrame with the imputed time slot columns
df[time_slot_cols] = df_imputed_first_grouping[time_slot_cols]
print("✅ Original DataFrame updated with imputed time slot columns.")

✅ First grouping imputation (by region, city, province) completed.
✅ Original DataFrame updated with imputed time slot columns.


In [89]:
# Count the rows with any null values in the time slot columns
null_rows_count = df[time_slot_cols].isnull().any(axis=1).sum()
print(f"Number of rows with null values in time slot columns in df: {null_rows_count}")

Number of rows with null values in time slot columns in df: 6175


In [91]:
df[time_slot_cols].head(30)

,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,09:00,14:30,continuous,continuous,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,22:00,09:00,14:30,19:00,22:00,09:00,16:00,continuous,continuous
6,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00
7,Closed,Closed,Closed,Closed,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous
8,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:00,continuous,continuous,12:00,14:00,continuous,continuous,12:00,14:00,19:00,21:00,19:00,21:00,continuous,continuous,12:00,14:00,19:00,21:00
9,Closed,Closed,Closed,Closed,09:00,16:00,continuous,continuous,09:00,16:00,continuous,continuous,09:00,21:00,continuous,continuous,09:00,21:00,continuous,continuous,16:45,23:45,continuous,continuous,09:00,17:00,continuous,continuous


### KNN imputer - group by: region, province

In [93]:
# 1️⃣ Define the time slot columns (those starting with day abbreviations)
# (Assumed already defined in your session as: 
# time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]
# If not, uncomment the next line:
# time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# 2️⃣ Convert each time slot column to categorical, store the mapping, and convert to numeric codes
category_maps = {}
for col in time_slot_cols:
    df[col] = df[col].astype('category')
    # Store the mapping: code -> original string
    category_maps[col] = dict(enumerate(df[col].cat.categories))
    # Convert to numeric codes; missing values become -1, so we replace those with NaN
    df[col] = df[col].cat.codes.replace(-1, np.nan)

# 3️⃣ Prepare a container to collect the imputed results per group
imputed_parts = []

# 4️⃣ Group by 'region' and 'province' and impute within each group
for group_key, group_df in df.groupby(['region', 'province']):
    # Extract the subset of time slot columns for the current group
    group_time_data = group_df[time_slot_cols]
    
    # Check if the entire group is missing (i.e. all values in group_time_data are NaN)
    if group_time_data.isnull().all().all():
        # Leave the group untouched by appending the original (all-NaN) data
        imputed_parts.append(group_time_data)
    else:
        # Determine the number of neighbors as the smaller of 3 or the group size
        n_neighbors = min(3, len(group_df))
        imputer = KNNImputer(n_neighbors=n_neighbors)
        # Apply KNN imputation on the group's time data
        imputed_array = imputer.fit_transform(group_time_data)
        # Create a DataFrame from the imputed result with the same columns and index as the group
        imputed_df = pd.DataFrame(imputed_array, columns=time_slot_cols, index=group_df.index)
        imputed_parts.append(imputed_df)

# 5️⃣ Recombine all imputed groups into one DataFrame, sorted by index
df_imputed_second_grouping = pd.concat(imputed_parts).sort_index()

# 6️⃣ Decode the numeric codes back to the original string values using the stored mappings
for col in time_slot_cols:
    mapping = category_maps[col]
    df_imputed_second_grouping[col] = df_imputed_second_grouping[col].round().astype('Int64').map(mapping)

print("✅ Second grouping imputation (by region, province) completed.")

# 7️⃣ Update the original DataFrame with the imputed time slot columns
df[time_slot_cols] = df_imputed_second_grouping[time_slot_cols]
print("✅ Original DataFrame updated with second imputed time slot columns.")

✅ Second grouping imputation (by region, province) completed.
✅ Original DataFrame updated with second imputed time slot columns.


In [95]:
# Count the rows with any null values in the time slot columns
null_rows_count = df[time_slot_cols].isnull().any(axis=1).sum()
print(f"Number of rows with null values in time slot columns in df: {null_rows_count}")

Number of rows with null values in time slot columns in df: 4


In [97]:
df[time_slot_cols].head(30)

,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd
0,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45
1,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45
2,14:30,19:30,23:00,23:30,11:55,16:45,22:00,23:00,11:45,16:40,21:30,22:50,11:15,15:30,20:30,22:40,11:00,15:15,20:15,22:40,11:40,16:15,21:00,22:50,12:45,18:30,23:01,23:30
3,13:30,18:30,23:00,23:30,11:55,16:30,22:00,23:00,11:50,16:00,21:00,23:00,11:40,15:30,20:30,22:40,11:30,15:00,20:00,22:30,11:50,15:45,20:30,22:45,13:45,19:00,23:01,23:45
4,12:30,18:45,23:00,23:30,11:45,16:40,22:45,23:15,11:40,16:30,22:00,23:02,11:00,16:00,22:00,23:00,11:00,15:30,21:00,22:50,11:40,16:15,22:00,23:00,12:15,18:30,23:15,23:45
5,09:00,14:30,continuous,continuous,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,22:00,09:00,14:30,19:00,22:00,09:00,16:00,continuous,continuous
6,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00
7,Closed,Closed,Closed,Closed,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous
8,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:00,continuous,continuous,12:00,14:00,continuous,continuous,12:00,14:00,19:00,21:00,19:00,21:00,continuous,continuous,12:00,14:00,19:00,21:00
9,Closed,Closed,Closed,Closed,09:00,16:00,continuous,continuous,09:00,16:00,continuous,continuous,09:00,21:00,continuous,continuous,09:00,21:00,continuous,continuous,16:45,23:45,continuous,continuous,09:00,17:00,continuous,continuous


### KNN imputer - group by: region

In [102]:
# 1️⃣ Define the time slot columns (those starting with day abbreviations)
# (Assumed already defined in your session as:
# time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]
# If not, uncomment the next line:
# time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]

# 2️⃣ Convert each time slot column to categorical, store the mapping, and convert to numeric codes
category_maps = {}
for col in time_slot_cols:
    df[col] = df[col].astype('category')
    # Store the mapping: code -> original string
    category_maps[col] = dict(enumerate(df[col].cat.categories))
    # Convert to numeric codes; missing values become -1, so we replace those with NaN
    df[col] = df[col].cat.codes.replace(-1, np.nan)

# 3️⃣ Prepare a container to collect the imputed results per group
imputed_parts = []

# 4️⃣ Group by 'region' and 'province' and impute within each group
for group_key, group_df in df.groupby(['region', 'province']):
    # Extract the subset of time slot columns for the current group
    group_time_data = group_df[time_slot_cols]
    
    # Check if the entire group is missing (i.e. all values in group_time_data are NaN)
    if group_time_data.isnull().all().all():
        # Leave the group untouched by appending the original (all-NaN) data
        imputed_parts.append(group_time_data)
    else:
        # Determine the number of neighbors as the smaller of 3 or the group size
        n_neighbors = min(3, len(group_df))
        imputer = KNNImputer(n_neighbors=n_neighbors)
        # Apply KNN imputation on the group's time data
        imputed_array = imputer.fit_transform(group_time_data)
        # Create a DataFrame from the imputed result with the same columns and index as the group
        imputed_df = pd.DataFrame(imputed_array, columns=time_slot_cols, index=group_df.index)
        imputed_parts.append(imputed_df)

# 5️⃣ Recombine all imputed groups into one DataFrame, sorted by index
df_imputed_third_grouping = pd.concat(imputed_parts).sort_index()

# 6️⃣ Decode the numeric codes back to the original string values using the stored mappings
for col in time_slot_cols:
    mapping = category_maps[col]
    df_imputed_third_grouping[col] = df_imputed_third_grouping[col].round().astype('Int64').map(mapping)

print("✅ Third grouping imputation (by region, province) completed.")

# 7️⃣ Update the original DataFrame with the imputed time slot columns from the third grouping
df[time_slot_cols] = df_imputed_third_grouping[time_slot_cols]
print("✅ Original DataFrame updated with third imputed time slot columns.")

✅ Third grouping imputation (by region, province) completed.
✅ Original DataFrame updated with third imputed time slot columns.


In [104]:
# Count the rows with any null values in the time slot columns
null_rows_count = df[time_slot_cols].isnull().any(axis=1).sum()
print(f"Number of rows with null values in time slot columns in df: {null_rows_count}")

Number of rows with null values in time slot columns in df: 4


### KNN imputer - global imputation

In [109]:
# --- Global Imputation on time slot columns ---

# Make sure time_slot_cols and category_maps are already defined from previous steps.
# If not, define them as follows:
# time_slot_cols = [col for col in df.columns if any(day in col for day in ['mon_', 'tue_', 'wed_', 'thu_', 'fri_', 'sat_', 'sun_'])]
#
# category_maps = {}
# for col in time_slot_cols:
#     df[col] = df[col].astype('category')
#     category_maps[col] = dict(enumerate(df[col].cat.categories))
#     df[col] = df[col].cat.codes.replace(-1, np.nan)

# --- Ensure the columns are in numeric code form ---
for col in time_slot_cols:
    # Re-convert to category and then to numeric codes, if needed.
    df[col] = df[col].astype('category').cat.codes.replace(-1, np.nan)

# --- Apply global KNN imputation ---
global_imputer = KNNImputer(n_neighbors=3)
imputed_global_array = global_imputer.fit_transform(df[time_slot_cols])
df_imputed_global = pd.DataFrame(imputed_global_array, columns=time_slot_cols, index=df.index)

# --- Decode the numeric codes back to the original string values ---
for col in time_slot_cols:
    mapping = category_maps[col]
    df_imputed_global[col] = df_imputed_global[col].round().astype('Int64').map(mapping)

# --- Update the original DataFrame with the globally imputed time slot columns ---
df[time_slot_cols] = df_imputed_global[time_slot_cols]

print("✅ Global imputation completed and original DataFrame updated.")

✅ Global imputation completed and original DataFrame updated.


In [111]:
# Count the rows with any null values in the time slot columns
null_rows_count = df[time_slot_cols].isnull().any(axis=1).sum()
print(f"Number of rows with null values in time slot columns in df: {null_rows_count}")

Number of rows with null values in time slot columns in df: 0


In [113]:
df[time_slot_cols].head(30)

,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd
0,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45
1,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,16:30,20:30,22:50,15:00,19:30,23:01,23:45
2,14:30,19:30,23:00,23:30,11:55,16:45,22:00,23:00,11:45,16:40,21:30,22:50,11:15,15:30,20:30,22:40,11:00,15:15,20:15,22:40,11:40,16:15,21:00,22:50,12:45,18:30,23:01,23:30
3,13:30,18:30,23:00,23:30,11:55,16:30,22:00,23:00,11:50,16:00,21:00,23:00,11:40,15:30,20:30,22:40,11:30,15:00,20:00,22:30,11:50,15:45,20:30,22:45,13:45,19:00,23:01,23:45
4,12:30,18:45,23:00,23:30,11:45,16:40,22:45,23:15,11:40,16:30,22:00,23:02,11:00,16:00,22:00,23:00,11:00,15:30,21:00,22:50,11:40,16:15,22:00,23:00,12:15,18:30,23:15,23:45
5,09:00,14:30,continuous,continuous,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,21:30,09:00,14:30,19:00,22:00,09:00,14:30,19:00,22:00,09:00,16:00,continuous,continuous
6,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00,12:00,14:30,18:30,22:00
7,Closed,Closed,Closed,Closed,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous,10:00,14:00,continuous,continuous
8,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,12:00,14:00,continuous,continuous,12:00,14:00,continuous,continuous,12:00,14:00,19:00,21:00,19:00,21:00,continuous,continuous,12:00,14:00,19:00,21:00
9,Closed,Closed,Closed,Closed,09:00,16:00,continuous,continuous,09:00,16:00,continuous,continuous,09:00,21:00,continuous,continuous,09:00,21:00,continuous,continuous,16:45,23:45,continuous,continuous,09:00,17:00,continuous,continuous


## rest of group C

In [118]:
# Define a mapping for Group C columns and their fill values
group_c_fill_map = {
    'awards': '0 or unknown',
    'special_diets': 'Missing',
    'features': 'Missing',
    'price_range': 'Unknown'
}

# Loop over the mapping and fill missing values for each column if it exists in the DataFrame
for col, fill_value in group_c_fill_map.items():
    if col in df.columns:
        df[col] = df[col].fillna(fill_value)

print("✅ Missing values in Group C columns have been filled.")

✅ Missing values in Group C columns have been filled.


In [124]:
# Count total number of missing (null) values in the specified columns
null_count = df[['awards', 'special_diets', 'features', 'price_range']].isnull().sum().sum()
print(f"Total number of null values in Group C columns: {null_count}")

Total number of null values in Group C columns: 0


# null handling in group D

## open_hours_per_week

In [132]:
def compute_daily_hours(row, day):
    """Compute total working hours for one day from its two shifts."""
    shift1 = 0.0
    shift2 = 0.0
    
    # First shift: e.g. "mon_open_1st" and "mon_close_1st"
    open1 = row.get(f"{day}_open_1st")
    close1 = row.get(f"{day}_close_1st")
    if pd.notnull(open1) and pd.notnull(close1) and open1 not in ["Closed", "continuous"] and close1 not in ["Closed", "continuous"]:
        try:
            t_open1 = pd.to_datetime(open1, format="%H:%M", errors='coerce')
            t_close1 = pd.to_datetime(close1, format="%H:%M", errors='coerce')
            if pd.notnull(t_open1) and pd.notnull(t_close1):
                shift1 = (t_close1 - t_open1).total_seconds() / 3600
                # Adjust for overnight shifts (if negative difference, add 24 hours)
                if shift1 < 0:
                    shift1 += 24
        except Exception:
            shift1 = 0.0

    # Second shift: e.g. "mon_open_2nd" and "mon_close_2nd"
    open2 = row.get(f"{day}_open_2nd")
    close2 = row.get(f"{day}_close_2nd")
    if pd.notnull(open2) and pd.notnull(close2) and open2 not in ["Closed", "continuous"] and close2 not in ["Closed", "continuous"]:
        try:
            t_open2 = pd.to_datetime(open2, format="%H:%M", errors='coerce')
            t_close2 = pd.to_datetime(close2, format="%H:%M", errors='coerce')
            if pd.notnull(t_open2) and pd.notnull(t_close2):
                shift2 = (t_close2 - t_open2).total_seconds() / 3600
                if shift2 < 0:
                    shift2 += 24
        except Exception:
            shift2 = 0.0

    return shift1 + shift2

def compute_weekly_hours(row):
    """Sum daily working hours across the week."""
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    total_hours = sum(compute_daily_hours(row, day) for day in days)
    return total_hours

# Use the computed weekly hours to fill missing values in the original column.
# Replace "open_hours_per_week" with the name of your original numeric column.
df["open_hours_per_week"].fillna(df.apply(compute_weekly_hours, axis=1), inplace=True)

print("✅ Nulls in 'open_hours_per_week' have been filled using computed weekly hours.")

✅ Nulls in 'open_hours_per_week' have been filled using computed weekly hours.


In [133]:
# Count total number of missing (null) values in the specified columns
null_count = df['open_hours_per_week'].isnull().sum().sum()
print(f"Total number of null values in open_hours_per_week: {null_count}")

Total number of null values in open_hours_per_week: 0


## open_days_per_week

In [137]:
def compute_open_days(row):
    """
    Computes the number of days a restaurant is open in a week.
    
    For each day (mon, tue, wed, thu, fri, sat, sun), if either the first 
    or second open time is not missing and not equal to "Closed", then the day is counted as open.
    """
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    open_count = 0
    for day in days:
        # Get the open time values for the day
        open1 = row.get(f"{day}_open_1st")
        open2 = row.get(f"{day}_open_2nd")
        # Count the day as open if at least one shift has a valid open time (not "Closed" and not NaN)
        if ((pd.notnull(open1) and open1 != "Closed") or 
            (pd.notnull(open2) and open2 != "Closed")):
            open_count += 1
    return open_count

# Update (or create) the 'open_days_per_week' column in the original DataFrame
df['open_days_per_week'] = df.apply(compute_open_days, axis=1)

print("✅ Computed 'open_days_per_week' and updated the original DataFrame.")

✅ Computed 'open_days_per_week' and updated the original DataFrame.


In [138]:
# Count total number of missing (null) values in the specified columns
null_count = df['open_days_per_week'].isnull().sum().sum()
print(f"Total number of null values in open_days_per_week: {null_count}")

Total number of null values in open_days_per_week: 0


## working_shifts_per_week

In [142]:
def compute_shifts_per_week(row):
    """
    Compute the number of working shifts in a week.
    For each day:
      - Count the first shift as 1 if both open and close times are provided and not "Closed".
      - For the second shift:
          - If it's marked as "continuous", assume the restaurant does not have a separate second shift.
          - Otherwise, if valid times are provided and not "Closed", count it as an additional shift.
    """
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    total_shifts = 0
    for day in days:
        # Retrieve values for first and second shifts
        open1 = row.get(f"{day}_open_1st")
        close1 = row.get(f"{day}_close_1st")
        open2 = row.get(f"{day}_open_2nd")
        close2 = row.get(f"{day}_close_2nd")
        
        # Initialize shift count for the day
        day_shifts = 0
        
        # First shift: count if open1 and close1 are present and not "Closed"
        if pd.notnull(open1) and pd.notnull(close1) and open1 != "Closed" and close1 != "Closed":
            day_shifts += 1
        
        # Second shift:
        if pd.notnull(open2) and pd.notnull(close2):
            # If either value is "continuous" (case-insensitive), assume it's not a separate shift.
            if str(open2).strip().lower() == "continuous" or str(close2).strip().lower() == "continuous":
                # If the first shift was not recorded (i.e. day_shifts is 0), then consider it as 1 shift.
                if day_shifts == 0:
                    day_shifts = 1
                # Otherwise, do not add an additional shift.
            else:
                # Otherwise, if the second shift times are valid and not "Closed", count it as another shift.
                if open2 != "Closed" and close2 != "Closed":
                    day_shifts += 1
        # Add the day's shifts to the total.
        total_shifts += day_shifts
    return total_shifts

# Compute the working shifts per week for each restaurant and update the original DataFrame.
df["working_shifts_per_week"] = df.apply(compute_shifts_per_week, axis=1)

print("✅ 'working_shifts_per_week' has been computed and updated in the original DataFrame.")

✅ 'working_shifts_per_week' has been computed and updated in the original DataFrame.


In [144]:
# Count total number of missing (null) values in the specified columns
null_count = df['working_shifts_per_week'].isnull().sum().sum()
print(f"Total number of null values in working_shifts_per_week: {null_count}")

Total number of null values in working_shifts_per_week: 0


## rank and total resturants

In [147]:
df[['rank', 'total_restaurants_in_city']] = df[['rank', 'total_restaurants_in_city']].fillna('Missing')
print("✅ Nulls in 'rank' and 'total_restaurants_in_city' have been filled with 'Missing'.")

✅ Nulls in 'rank' and 'total_restaurants_in_city' have been filled with 'Missing'.


# check if there is still columns with nulls

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155288 entries, 0 to 155287
Columns: 280 entries, region to total_restaurants_in_city
dtypes: float64(1), int64(2), object(277)
memory usage: 331.7+ MB


In [152]:
df.head()

,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,successful,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
0,nouvelle aquitaine,haute vienne,saint jouvent,claimed,0 or unknown,"cheap eats, french",€,Unknown,"lunch, dinner",french,Missing,"reservations, seating, wheelchair accessible, serves alcohol, accepts credit cards, table service",N,N,N,7,46.066667,14,english,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,0 or unknown,N,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,1

In [154]:
# Get the list of columns
cols = df.columns.tolist()

# Check if 'successful' exists and move it to the front
if 'successful' in cols:
    cols.insert(0, cols.pop(cols.index('successful')))
    df = df[cols]

print("✅ 'successful' column moved to the first position.")

✅ 'successful' column moved to the first position.


In [156]:
df.head()

,successful,region,province,city,claimed,awards,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,open_days_per_week,open_hours_per_week,working_shifts_per_week,default_language,accepts american express,accepts credit cards,accepts discover,accepts mastercard,accepts visa,afghani,african,after hours,albanian,algerian,american,apulian,arabic,argentinian,armenian,asian,assyrian,australian,austrian,azerbaijani,bahamian,bakeries,balti,bangladeshi,bar,barbecue,basque,beach,beer restaurants,beijing cuisine,beijing specialties,belgian,brazilian,breakfast,brew pub,british,brunch,buffet,burmese,byob,cafe,cajun creole,calabrian,cambodian,campania,canadian,cantonese,caribbean,cash only,catalan,caucasian,central american,central asian,central european,central italian,cheap eats,chilean,chinese,colombian,contemporary,costa rican,croatian,cuban,czech,danish,deli,delivery,delivery only,dessert,digital payments,dine with a local chef,diner,dining bars,dinner,dog friendly,drinks,drive thru,dutch,eastern european,ecuadorean,egyptian,emilian,ethiopian,european,family style,fast food,fine dining,free off street parking,free wifi,french,fruit parlours,fujian,full bar,fusion,gastropub,georgian,german,gift cards available,gluten free options,greek,grill,guangxi,guatemalan,halal,hawaiian,healthy,highchairs available,hokkaido cuisine,hong kong,hubei,hunan,hungarian,imperial chinese,indian,indonesian,international,irish,israeli,italian,jamaican,japanese,japanese fusion,japanese sweets parlour,jazz bar,kaiseki,kappo,korean,kosher,kyoto cuisine,kyushu cuisine,latin,latvian,lazio,lebanese,ligurian,live music,lombard,lunch,malaysian,medicinal foods,mediterranean,mexican,mid range,middle eastern,mongolian,moroccan,native american,neapolitan,nepalese,new zealand,nigerian,non smoking restaurants,northeastern chinese,northern italian,northwestern chinese,norwegian,outdoor seating,pakistani,parking available,persian,peruvian,philippine,pizza,playgrounds,polish,polynesian,portuguese,private dining,pub,puerto rican,quick bites,reservations,romagna,romana,romanian,russian,salvadoran,sardinian,scandinavian,scottish,seafood,seating,serves alcohol,shandong,shanghai,sicilian,singaporean,slovenian,soups,south american,southern italian,southwestern,spanish,speciality food market,sports bars,sri lankan,steakhouse,street food,street parking,sushi,swedish,swiss,szechuan,table service,taiwanese,takeout,television,thai,tibetan,tunisian,turkish,tuscan,ukrainian,uzbek,valet parking,validated parking,vegan options,vegetarian friendly,venezuelan,vietnamese,waterfront,welsh,wheelchair accessible,wine and beer,wine bar,xinjiang,yunnan,zhejiang,award_count,moved_location,mon_open_1st,mon_close_1st,mon_open_2nd,mon_close_2nd,tue_open_1st,tue_close_1st,tue_open_2nd,tue_close_2nd,wed_open_1st,wed_close_1st,wed_open_2nd,wed_close_2nd,thu_open_1st,thu_close_1st,thu_open_2nd,thu_close_2nd,fri_open_1st,fri_close_1st,fri_open_2nd,fri_close_2nd,sat_open_1st,sat_close_1st,sat_open_2nd,sat_close_2nd,sun_open_1st,sun_close_1st,sun_open_2nd,sun_close_2nd,rank,total_restaurants_in_city
0,N,nouvelle aquitaine,haute vienne,saint jouvent,claimed,0 or unknown,"cheap eats, french",€,Unknown,"lunch, dinner",french,Missing,"reservations, seating, wheelchair accessible, serves alcohol, accepts credit cards, table service",N,N,N,7,46.066667,14,english,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,0 or unknown,N,13:30,19:15,22:45,23:30,11:45,16:40,21:30,23:00,11:45,16:30,21:00,23:00,11:30,15:45,20:15,22:45,11:00,15:45,20:00,22:40,11:50,1

In [158]:
# Compute a summary of nulls per column
null_summary = df.isnull().sum()

# Print only columns with missing values
print("Columns with missing values:")
print(null_summary[null_summary > 0])

# Also print the total number of nulls in the DataFrame
total_nulls = df.isnull().sum().sum()
print(f"Total number of null values in the DataFrame: {total_nulls}")

Columns with missing values:
price_level    29805
dtype: int64
Total number of null values in the DataFrame: 29805


# price level

In [161]:
df['price_level'].unique()

array(['€', '€€-€€€', nan, '€€€€'], dtype=object)

In [167]:
mode_val = df['price_level'].mode()[0]
df['price_level'] = df['price_level'].fillna(mode_val)
print(f"✅ Price level nulls filled with mode: {mode_val}")

✅ Price level nulls filled with mode: €€-€€€


In [169]:
# Compute a summary of nulls per column
null_summary = df.isnull().sum()

# Print only columns with missing values
print("Columns with missing values:")
print(null_summary[null_summary > 0])

# Also print the total number of nulls in the DataFrame
total_nulls = df.isnull().sum().sum()
print(f"Total number of null values in the DataFrame: {total_nulls}")

Columns with missing values:
Series([], dtype: int64)
Total number of null values in the DataFrame: 0
